In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
all_cases = ['DUCT_1100','DUCT_1150','DUCT_1250','DUCT_1300','DUCT_1350','DUCT_1400','DUCT_1500','DUCT_1600','DUCT_1800',
         'DUCT_2000','DUCT_2205','DUCT_2400','DUCT_2600','DUCT_2900','DUCT_3200','DUCT_3500',
         'PHLL_case_0p5','PHLL_case_0p8','PHLL_case_1p0','PHLL_case_1p2','PHLL_case_1p5',
         'BUMP_h20','BUMP_h26','BUMP_h31','BUMP_h38','BUMP_h42',
         'CNDV_12600','CNDV_20580','CBFS_13700']
cases = ['PHLL_case_0p5','PHLL_case_0p8','PHLL_case_1p0','PHLL_case_1p2','PHLL_case_1p5']
#Convenient functions for loading dataset
def loadCombinedArray(cases,field):
    data = np.concatenate([np.load('/home/haoyan/Desktop/ML-CFD/dataset/archive/'+dataset+'/'+dataset+'_'+case+'_'+field + '.npy') for case in cases])
    return data
         
def loadLabels(cases,field):
    data = np.concatenate([np.load('/home/haoyan/Desktop/ML-CFD/dataset/archive/'+'labels/'+case+'_'+field + '.npy') for case in cases])
    return data
         
# Select RANS model
dataset = 'komega'
print('Loading features and labels from the dataset: '+ dataset)
         
Tensors = loadCombinedArray(cases,'Tensors')
print('Shape of basis tensor array: '+str(Tensors.shape))

Invariants = loadCombinedArray(cases,'I')
print('Shape of invariant features array: '+str(Invariants.shape))

#Load the additional scalars (N,5): 
#    q[:,0] = Ratio of excess rotation to strain rate,
#    q[:,1] = Wall-distance based Reynolds number, 
#    q[:,2] = Ratio of turbulent time scale to mean strain time scale
#    q[:,3] = Ratio of total Reynolds stress to 1/2 * normal Reynolds stress (TKE)
Scalars = loadCombinedArray(cases,'q')
print('Shape of scalar features array: '+str(Scalars.shape))

#Combine the invariants and scalars to form a feature set
Features = np.column_stack((Invariants,Scalars))
print('Shape of combined features array: '+str(Features.shape))

#Optional: remove outliers based on the number of standard deviations away from the mean. 
#Note: must be careful, as there are naturally large variations in flow features. Even a 5*stdev critera removes many valid near-wall points.
def remove_outliers(Features):
    stdev = np.std(Features,axis=0)
    means = np.mean(Features,axis=0)
    ind_drop = np.empty(0)
    for i in range(len(Features[0,:])):
        ind_drop = np.concatenate((ind_drop,np.where((Features[:,i]>means[i]+5*stdev[i]) | (Features[:,i]<means[i]-5*stdev[i]))[0]))
    return ind_drop.astype(int)

outlier_removal_switch = 0
if outlier_removal_switch == 1:
    outlier_index = remove_outliers(Features)
    print('Found '+str(len(outlier_index))+' outliers in the input feature set')
    Features = np.delete(Features,outlier_index,axis=0)
    Tensors = np.delete(Tensors,outlier_index,axis=0)
    Labels = np.delete(Labels,outlier_index,axis=0)
    
#Load the label set from DNS/LES:
Labels = loadLabels(cases,'b')
#If desired, reshape the 3x3 symmetric anisotropy tensor into a 1x6 vector
Labels = np.delete(Labels.reshape((len(Labels),9)),[3,6,7],axis=1)
print('Shape of DNS/LES labels array: '+str(Labels.shape))

# Split the datasets into training and validation
indices = np.arange(Features.shape[0])
input_shape = Features.shape[1]

x_train, x_val, y_train, y_val, ind_train, ind_val = train_test_split(Features, Labels, indices, test_size=0.2, random_state=10, shuffle=True)

basis_train = Tensors[ind_train]
basis_val = Tensors[ind_val]

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

print(' ')
print('Training features:')
print(x_train.shape)
print('Training tensor basis:')
print(basis_train.shape)
print('Training labels:')
print(y_train.shape)
print(' ')
print('Validation features:')
print(x_val.shape)
print('Validation tensor basis:')
print(basis_val.shape)
print('Validation labels:')
print(y_val.shape)
print(' ')

Loading features and labels from the dataset: komega
Shape of basis tensor array: (73754, 10, 3, 3)
Shape of invariant features array: (73754, 47)
Shape of scalar features array: (73754, 4)
Shape of combined features array: (73754, 51)
Shape of DNS/LES labels array: (73754, 6)
 
Training features:
(59003, 51)
Training tensor basis:
(59003, 10, 3, 3)
Training labels:
(59003, 6)
 
Validation features:
(14751, 51)
Validation tensor basis:
(14751, 10, 3, 3)
Validation labels:
(14751, 6)
 


In [5]:
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()

#The model has two inputs, a set of input features with a learned mapping, and the tensor basis layer
input_layer = keras.layers.Input(shape=(input_shape),name ='input_layer')
input_tensor_basis = keras.layers.Input(shape=(10,3,3),name='Tensor_input_layer')

#Hidden layer definition
hidden1 = keras.layers.Dense(30,name='Hidden1', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(input_layer)
hidden2 = keras.layers.Dense(30,name='Hidden2', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden1)
hidden3 = keras.layers.Dense(30,name='Hidden3', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden2)
hidden4 = keras.layers.Dense(30,name='Hidden4', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden3)
hidden5 = keras.layers.Dense(30,name='Hidden5', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden4)
hidden6 = keras.layers.Dense(30,name='Hidden6', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden5)
hidden7 = keras.layers.Dense(30,name='Hidden7', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden6)
hidden8 = keras.layers.Dense(30,name='Hidden8', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "LeakyReLU")(hidden7)


#The layer of gn, which are coefficients for each of the ten Tn
gn = keras.layers.Dense(10,name='gn', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-4), activation = "selu")(hidden8)

#Multiply the gn by Tn, with the output being the anisotropy tensor
shaped = keras.layers.Reshape((10,1,1),name='Shape_for_dot_product')(gn)
merge = keras.layers.Dot(axes=1, name='Dot_product')([shaped,input_tensor_basis])

#Reshape the output anisotropy tensor, and trim out duplicate values (it is a symmetric matrix). The end result is a 6 component vector.
shaped_output = keras.layers.Reshape((9,1),name='Shaped_output')(merge)
trimmed_output1 = keras.layers.Lambda(lambda x : x[:,0])(shaped_output)
trimmed_output2 = keras.layers.Lambda(lambda x : x[:,1])(shaped_output)
trimmed_output3 = keras.layers.Lambda(lambda x : x[:,2])(shaped_output)
trimmed_output4 = keras.layers.Lambda(lambda x : x[:,4])(shaped_output)
trimmed_output5 = keras.layers.Lambda(lambda x : x[:,5])(shaped_output)
trimmed_output6 = keras.layers.Lambda(lambda x : x[:,8])(shaped_output)
merged_output = tf.keras.layers.Concatenate()([trimmed_output1,trimmed_output2,trimmed_output3,trimmed_output4,trimmed_output5,trimmed_output6])


model=keras.Model(inputs=[input_layer, input_tensor_basis], outputs=[merged_output])


optimizer = tf.keras.optimizers.Nadam(learning_rate = 2E-5, clipnorm=1000)
model.compile(optimizer,loss='mse',metrics=['mae', 'mse'])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 51)]         0           []                               
                                                                                                  
 Hidden1 (Dense)                (None, 30)           1560        ['input_layer[0][0]']            
                                                                                                  
 Hidden2 (Dense)                (None, 30)           930         ['Hidden1[0][0]']                
                                                                                                  
 Hidden3 (Dense)                (None, 30)           930         ['Hidden2[0][0]']                
                                                                                              

In [8]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.2,patience=10,verbose=0,mode="auto",min_delta=0.0001,cooldown=0,min_lr=0)
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0,patience=40,verbose=0,mode="auto",baseline=None,restore_best_weights=True)
history = model.fit([x_train,basis_train],y_train,batch_size=1000,epochs=1000,validation_data = ([x_val,basis_val],y_val),verbose=1)

Epoch 1/1000


2022-06-22 11:06:10.834303: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12036612 exceeds 10% of free system memory.
2022-06-22 11:06:10.847513: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 21241080 exceeds 10% of free system memory.


60/60 [==============================] - 3s 11ms/step - loss: 1.5947 - mae: 0.3462 - mse: 1.3578 - val_loss: 1.1432 - val_mae: 0.2799 - val_mse: 0.9064
Epoch 2/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.9452 - mae: 0.2360 - mse: 0.7085 - val_loss: 0.7460 - val_mae: 0.2061 - val_mse: 0.5093
Epoch 3/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.6808 - mae: 0.1890 - mse: 0.4441 - val_loss: 0.5781 - val_mae: 0.1770 - val_mse: 0.3415
Epoch 4/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.5575 - mae: 0.1688 - mse: 0.3209 - val_loss: 0.4910 - val_mae: 0.1621 - val_mse: 0.2545
Epoch 5/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.4881 - mae: 0.1574 - mse: 0.2517 - val_loss: 0.4388 - val_mae: 0.1527 - val_mse: 0.2025
Epoch 6/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.4432 - mae: 0.1494 - mse: 0.2069 - val_loss: 0.4043 - val_mae: 0.1458 - val_mse: 0.1680
Epoch 7/1000
60/60 [========

Epoch 51/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2464 - mae: 0.0633 - mse: 0.0168 - val_loss: 0.2439 - val_mae: 0.0622 - val_mse: 0.0145
Epoch 52/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2458 - mae: 0.0628 - mse: 0.0165 - val_loss: 0.2434 - val_mae: 0.0617 - val_mse: 0.0142
Epoch 53/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2453 - mae: 0.0624 - mse: 0.0161 - val_loss: 0.2430 - val_mae: 0.0613 - val_mse: 0.0139
Epoch 54/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2448 - mae: 0.0621 - mse: 0.0158 - val_loss: 0.2425 - val_mae: 0.0610 - val_mse: 0.0137
Epoch 55/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2443 - mae: 0.0618 - mse: 0.0155 - val_loss: 0.2421 - val_mae: 0.0606 - val_mse: 0.0134
Epoch 56/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2438 - mae: 0.0613 - mse: 0.0152 - val_loss: 0.2417 - val_mae: 0.0602 - val_mse: 0.0132
Epoch 57/1

Epoch 101/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2235 - mae: 0.0486 - mse: 0.0081 - val_loss: 0.2228 - val_mae: 0.0479 - val_mse: 0.0077
Epoch 102/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2230 - mae: 0.0484 - mse: 0.0080 - val_loss: 0.2223 - val_mae: 0.0477 - val_mse: 0.0076
Epoch 103/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2225 - mae: 0.0482 - mse: 0.0080 - val_loss: 0.2218 - val_mae: 0.0475 - val_mse: 0.0075
Epoch 104/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.2219 - mae: 0.0480 - mse: 0.0079 - val_loss: 0.2213 - val_mae: 0.0473 - val_mse: 0.0075
Epoch 105/1000
60/60 [==============================] - 0s 7ms/step - loss: 0.2214 - mae: 0.0478 - mse: 0.0078 - val_loss: 0.2208 - val_mae: 0.0471 - val_mse: 0.0074
Epoch 106/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.2208 - mae: 0.0476 - mse: 0.0077 - val_loss: 0.2202 - val_mae: 0.0470 - val_mse: 0.0073
Epoc

...

60/60 [==============================] - 0s 6ms/step - loss: 0.0072 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 592/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0072 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 593/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0072 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 594/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0072 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0381 - val_mse: 0.0050
Epoch 595/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0072 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0383 - val_mse: 0.0050
Epoch 596/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0072 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 597/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 690/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 691/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 692/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 693/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 694/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0381 - val_mse: 0.0050
Epoch 695/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 788/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 789/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0381 - val_mse: 0.0050
Epoch 790/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 791/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 792/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 793/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 886/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 887/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 888/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 889/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 890/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 891/1000
60/6

60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 984/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0386 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0380 - val_mse: 0.0050
Epoch 985/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0380 - val_mse: 0.0050
Epoch 986/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0379 - val_mse: 0.0050
Epoch 987/1000
60/60 [==============================] - 0s 6ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0070 - val_mae: 0.0387 - val_mse: 0.0050
Epoch 988/1000
60/60 [==============================] - 0s 5ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052 - val_loss: 0.0069 - val_mae: 0.0378 - val_mse: 0.0050
Epoch 989/1000
60/6

In [10]:
model.evaluate([x_train,basis_train],y_train)

  57/1844 [..............................] - ETA: 3s - loss: 0.0069 - mae: 0.0379 - mse: 0.0049

2022-06-22 11:13:23.440361: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12036612 exceeds 10% of free system memory.


1844/1844 [==============================] - 3s 2ms/step - loss: 0.0071 - mae: 0.0385 - mse: 0.0052


[0.007075480185449123, 0.03854236379265785, 0.005173050798475742]

In [11]:
model.evaluate([x_val,basis_val],y_val)

461/461 [==============================] - 1s 2ms/step - loss: 0.0069 - mae: 0.0378 - mse: 0.0050


[0.0068856654688715935, 0.03784329071640968, 0.004983242135494947]

In [1]:
save = model.save("/home/haoyan/Desktop/ML-CFD/case.h5")

NameError: name 'model' is not defined